# Interim Notebook (Labs 4 + 5)
### Owen Yaggy
### 10/13/23
### MUSC H255

## Importing libraries:
All the libraries that could be needed. This is directly from the Encoding Music tutorial on Spotify functions on GitHub. As a result, not all of these libraries are currently being used. In addition, I was having issues with the community library, but since it isn't currently necessary, I removed that import statement for the time being.

Finally, I added my own input statement, `import config`. On my local machine, I have a file called config.py. Since I store all my code for this class in a GitHub repository, I want to avoid any unnecessary personal data in the code. My config.py file is on my .gitignore list, so my credentials stay private.

Unusual dependencies:
* Download spotify_tools.py from the Encoding Music GitHub
* A file, config.py, with variables for `client_id`, `client_secret`, and `user` stored as strings. These correspond to your public and private Spotify API keys, and your Spotify username, respectively, OR:
* Replace the usage of `config.client_id` with your private API key, and so on for the other variables. They are only used once, in the code block following the import statements code block.

In [11]:
import pandas as pd
import numpy as np
import random
import altair as alt
import plotly.graph_objects as go
import requests
import inspect
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import spotify_tools
import networkx as nx
import networkx.algorithms.community as nx_comm
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import plotly.express as px
import pyvis
from pyvis import network as net
from itertools import combinations
# from community import community_louvain
from copy import deepcopy
import time

import config   # A file I store on my local device containing my Spotify API credentials

Store the credentials based on the config.py and set up the spotipy library with those credentials.

**This is where you can replace the variable usage with your own API keys and username.** 

In [12]:
# storing the credentials:
#
CLIENT_ID = config.client_id
CLIENT_SECRET = config.client_secret
my_username = config.user

# instantiating the client.  This 'sp' version of the client is used repeatedly below
# source: Max Hilsdorf (https://towardsdatascience.com/how-to-create-large-music-datasets-using-spotipy-40e7242cc6a6)
client_credentials_manager = SpotifyClientCredentials(client_id=CLIENT_ID, client_secret=CLIENT_SECRET)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

Take a specific playlist, store it in a dataframe, and show the result.

In [13]:
creator_id = "lzsfootball"
playlist_id = "1hTUFxr380FuNNYHpynyzy"

# playlist_tracks(user_id: String, playlist_id: String): json_dict
playlist_tracks = pd.DataFrame(sp.user_playlist_tracks(creator_id, playlist_id))
playlist_tracks

,href,items,limit,next,offset,previous,total
0,https://api.spotify.com/v1/playlists/1hTUFxr38...,"{'added_at': '2023-10-06T17:42:39Z', 'added_by...",100,None,0,None,10
1,https://api.spotify.com/v1/playlists/1hTUFxr38...,"{'added_at': '2023-10-06T17:42:42Z', 'added_by...",100,None,0,None,10
2,https://api.spotify.com/v1/playlists/1hTUFxr38...,"{'added_at': '2023-10-06T17:43:22Z', 'added_by...",100,None,0,None,10
3,https://api.spotify.com/v1/playlists/1hTUFxr38...,"{'added_at': '2023-10-06T17:44:56Z', 'added_by...",100,None,0,None,10
4,https://api.spotify.com/v1/playlists/1hTUFxr38...,"{'added_at': '2023-10-06T17:45:31Z', 'added_by...",100,None,0,None,10
5,https://api.spotify.com/v1/playlists/1hTUFxr38...,"{'added_at': '2023-10-06T17:46:20Z', 'added_by...",100,None,0,None,10
6,https://api.spotify.com/v1/playlists/1hTUFxr38...,"{'added_at': '2023-10-06T17:47:14Z', 'added_by...",100,None,0,None,10
7,https://api.spotify.com/v1/playlists/1hTUFxr38...,"{'added_at': '2023-10-06T17:48:01Z', 'added_by...",100,None,0,None,10
8,https://api.spotify.com/v1/playlists/1hTUFxr38...,"{'added_at': '2023-10-06T17:50:00Z', 'added_by...",100,None,0,None,10
9,https://api.spotify.com/v1/playlists/1hTUFxr38...,"{'added_at': '2023-10-06T17:51:32Z', 'added_by...",100,None,0,None,10


Experiment: show a single song's name.
*Challenge:* Because the data returned by Spotify's API is so voluminous and complex, it's actually quite difficult to find out how to query a very specific piece of data.

In [14]:
sample_track = playlist_tracks.iloc[0]["items"]["track"]['name']
sample_track

'Dumb'

Show what one row in data looks like.

In [15]:
playlist_tracks.iloc[0]['items']

{'added_at': '2023-10-06T17:42:39Z',
 'added_by': {'external_urls': {'spotify': 'https://open.spotify.com/user/lzsfootball'},
  'href': 'https://api.spotify.com/v1/users/lzsfootball',
  'id': 'lzsfootball',
  'type': 'user',
  'uri': 'spotify:user:lzsfootball'},
 'is_local': False,
 'primary_color': None,
 'track': {'album': {'album_type': 'album',
   'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/6olE6TJLqED3rqDCT0FyPh'},
     'href': 'https://api.spotify.com/v1/artists/6olE6TJLqED3rqDCT0FyPh',
     'id': '6olE6TJLqED3rqDCT0FyPh',
     'name': 'Nirvana',
     'type': 'artist',
     'uri': 'spotify:artist:6olE6TJLqED3rqDCT0FyPh'}],
   'available_markets': ['AR',
    'AU',
    'AT',
    'BE',
    'BO',
    'BR',
    'BG',
    'CA',
    'CL',
    'CO',
    'CR',
    'CY',
    'CZ',
    'DK',
    'DO',
    'DE',
    'EC',
    'EE',
    'SV',
    'FI',
    'FR',
    'GR',
    'GT',
    'HN',
    'HK',
    'HU',
    'IS',
    'IE',
    'IT',
    'LV',
    'LT',
 

**First function:** Given the dataframe of tracks, return the Spotify song IDs.
It seems like this wasn't truly necessary, after discovering the function I was trying to build with this data already existed on the Encoding Music GitHub, but it was a useful experiment to try and get specific information from the dataframe.
However, this can now be used to query the results of the `analyze_playlist` function based on specific song ids.

In [16]:
def get_song_ids(tracks: pd.DataFrame) -> dict:     # Parameter is a dataframe, output is a dict
    """Returns the Spotify song IDs contained in the playlist dataframe, matched with the song titles to make them easy to identify"""
    song_ids = {}       # Empty dict to contain all outputted song ids
    for track in tracks['items']:           # Iterate through tracks in dataframe
        song_ids[track['track']['name']] = track['track']['id']          # add song id to list
    return song_ids    # return song ids

print(get_song_ids(playlist_tracks))

{'Dumb': '5gRcv46AMTrosmTOqrOV3Q', 'Creep': '70LcF31zb1H0PyJoS1Sx1r', 'Jerk': '065bY5WrLvI3NN9VCK0QLe', 'Son of a Bitch': '6Al4oITGudbYvcBzO65ah6', 'Dickhead': '25aNJcqUnZmEKBmPhW9I82', 'Snitch': '3sXaTInkrXe5YEaa6ZYr1S', 'Moron': '3OmhbyRDJDm1zhfVEV7d0w', 'Heartbreaker': '22ML0MuFKfw16WejbxsLOy', 'Jackass': '2UiUTpQkTCmE1OX6senl6j', 'Fucker': '2khc6kEG4eBvYN1GvR6R97'}


**Get music feature data!** This is needed to analyze the songs and determine similarity.

*This code is taken from the EncodingMusic GitHub, in spotify_tools.py, but MODIFIED as follows:*

* Previously, the index of every song in the dataframe was sequential (0, 1, 2, etc.)
* This made it hard for me to query a specific audio feature for a track
* So, I changed the code so that it the index becomes the track_id for a song
* This means you can query based on track_id, which makes it easier to find specific data for a song
* HOWEVER, I did not consider that playlists could contain several copies of the exact same song
* I believe this now might pose errors if creating a dataframe for a playlist that contains the same song more than once
* I'm not sure how I would want to resolve that

**Problems Encountered**

Ultimately, the difficult part of this lab has been figuring out how to work with Pandas. While ultimately the tasks are not overly complex, there are so many ways of retrieving data that the syntax for doing so is rather convoluted. I've encountered countless error messages so far related to querying data in the wrong way, which I did forget to record as I was encountering them. 

In [17]:
def analyze_playlist(creator, playlist_id, spotipy_client):
    playlist_features_list = ["artist", "album", "track_name",
                              "danceability", "energy", "key", "loudness", "mode", "speechiness",
                              "instrumentalness", "liveness", "valence", "tempo", "duration_ms", "time_signature"] # removed track_id
    playlist_df = pd.DataFrame(columns=playlist_features_list)

    playlist_features = {}

    playlist = spotipy_client.user_playlist_tracks(creator, playlist_id)["items"]
    for track in playlist:
        playlist_features["artist"] = track["track"]["album"]["artists"][0]["name"]
        playlist_features["album"] = track["track"]["album"]["name"]
        playlist_features["track_name"] = track["track"]["name"]
        playlist_features["track_id"] = track["track"]["id"]

        audio_features = spotipy_client.audio_features(playlist_features["track_id"])[0]
        for feature in playlist_features_list[3:]: # changed from 4 to 3 to fix error caused by removing track_id from list of features
            playlist_features[feature] = audio_features[feature]

        track_df = pd.DataFrame(playlist_features, index=[playlist_features['track_id']]) # changed index from automatically being 0 to being the track_id
        playlist_df = pd.concat([playlist_df, track_df], ignore_index=False) # changed ignore_index from True to False

    return playlist_df

I needed to test what how I could query the data returned from this function:

In [18]:
df = analyze_playlist(creator_id, playlist_id, sp)
print(df)
print(df.at['5gRcv46AMTrosmTOqrOV3Q', 'liveness'])
print(type(df.at['5gRcv46AMTrosmTOqrOV3Q', 'liveness']))
x = df.loc['5gRcv46AMTrosmTOqrOV3Q']
got = x.get('liveness')
print(got)

/var/folders/24/qxr9c06j19gcvdhrn3t51m000000gn/T/ipykernel_88185/3211627563.py:21: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  playlist_df = pd.concat([playlist_df, track_df], ignore_index=False) # changed ignore_index from True to False


                                              artist  \
5gRcv46AMTrosmTOqrOV3Q                       Nirvana   
70LcF31zb1H0PyJoS1Sx1r                     Radiohead   
065bY5WrLvI3NN9VCK0QLe                   Oliver Tree   
6Al4oITGudbYvcBzO65ah6                         Kyuss   
25aNJcqUnZmEKBmPhW9I82                     Kate Nash   
3sXaTInkrXe5YEaa6ZYr1S                     Lil Wayne   
3OmhbyRDJDm1zhfVEV7d0w             The Murderburgers   
22ML0MuFKfw16WejbxsLOy                   Pat Benatar   
2UiUTpQkTCmE1OX6senl6j                     Green Day   
2khc6kEG4eBvYN1GvR6R97  The Brian Jonestown Massacre   

                                           album      track_name  \
5gRcv46AMTrosmTOqrOV3Q                  In Utero            Dumb   
70LcF31zb1H0PyJoS1Sx1r               Pablo Honey           Creep   
065bY5WrLvI3NN9VCK0QLe         Ugly is Beautiful            Jerk   
6Al4oITGudbYvcBzO65ah6                    Wretch  Son of a Bitch   
25aNJcqUnZmEKBmPhW9I82            Made of B

# THE BIG ONE #

This is work towards my SpotiPy project.

While we ultimately aim to relate people, this code ultimately outputs a scalar telling you how similar 2 songs are.
The main function is `similarity_scalar(...)`:
 `similarity_scalar(song_id_1, song_id_2, playlist_dataframe)`
 
To explain how the value is computed, I can walk through the code:
1. First, in `similarity_dict(...)`:
    * Audio feature data is stored in a dictionary for each track
    * The difference between the values for each feature is computed
        * So, a small number for `valence` indicates 2 tracks have very similar valence, while a large number indicates very different valence
        * For scalar features (those measured from 1-0), this is a simple process
        * Some features just need to be translated to a different scale (e.g. `loudness` can be transposed from a -60 to 0 scale to a 0 to 1 scale)
        * For comparing keys, a more complicated algorithm is needed to account for similar keys (see `compare_keys(...)`)
    * The output is a dictionary where each entry is measures how similar a feature is, for example `'valence': 0.03` indicates similar valence
2. Then, in `similarity_scalar_from_dict(...)`:
    * The dictionary resulting from `similarity_dict(...)` is taken as an input
    * Every difference between similarities is summed
    * This number is divided by the total number of audio features (we use 9)
    * The result is a number between 0-1, where 0 represents identical, and 1 represents completely different
    * However, we want 1 to represent identical, and 0 to represent completely different, so we invert this scale (subtracting the number we got before from 1)
3. Finally, in `similarity_scalar(...)`:
    * This function is the exact same as `similarity_scalar_from_dict(...)`, it just takes song IDs and the playlist dataframe as inputs
    * This means you only need to use one function to get the data you want
    * It inputs the parameters into `similarity_dict(...)` and `similarity_scalar_from_dict(...)`, and returns the result

The functions `similarity_scalar_from_dict(...)` and `similarity_scalar(...)` have one more important characteristic:

### AUDIO FEATURE WEIGHTING ###

For various reasons, we might decide that not all every Spotify audio feature should be weighted equally when comparing tracks. For example, is `instrumentalness` really as important as `valence`?

While the `similarity_scalar` methods by default treat all audio features equally, you can use optional parameters to customize the weighting of each audio feature. The default weight is 1. So if you wanted to prioritize `valence`, and de-emphasize `mode`, you could write:
`similarity_scalar(song_id_1, song_id_2, playlist_dataframe, valence=3, mode=0)`
This would weight valence 3 times as much as every other audio feature. This can be a float, like 0.5, or even 0.

The code for this is somewhat unclean, because it uses error handling. Partially because I wanted to experiment with it, this method uses **kwargs. The code assumes the user has given a custom parameter for every audio feature, and handles the KeyError that occurs whenever they haven't. It works by multiplying the similarity difference for a feature by its weight before adding it to the total similarity difference for 2 tracks. The resulting sum is then divided by the sum of all the weights combined.

The better way to do this would be to include an optional, default parameter in the method definition for each possible audio feature. This is better because it eliminates the need for error handling, and, makes it possible to set type hints for weights, and gives more useful errors (if you try to set an audio feature that doesn't exist, you'll get an error, and if you set it to an invalid value, you'll know more easily).

#### Next Steps ####
The natural next step is writing a function to compare all the songs in a playlist to each other. However, since this increases the amount of data crunching by an order of magnitude, I am curious (if not concerned) about how long this might take, especially since our initial plan is comparing every song to each other in order to relate people. With 72 playlists * 10 songs = 720 songs, that's (720(720 + 1))/2 = 259,560 song comparisons, each of which has at least O(n^2) runtime. I'm curious how long that might take.

Since each playlist is a dataframe, we would also need to create a dataframe containing each playlist dataframe.

In [19]:
def compare_keys(key1: int, mode1: int, key2: int, mode2: int) -> float:
    """Returns whether the key of 2 tracks is similar based on the circle of fifths"""
    # Similar means similar audio feature values. Thus returning 0 means very similar, and 1 means very different.
    if key1 == key2 and mode1 == mode2:
        return 0.0
    if mode1 == 0 and key2 == (key1 + 3) % 12 and mode2 == 1:      # The modulo by 12 accounts for cases where key + 3 > 11
        return 0.0
    if mode1 == 1 and key2 == (key1 - 3) % 12 and mode2 == 0:      # The modulo by 12 accounts for cases where key - 3 < 0
        return 0.0
    return 1.0


def similarity_dict(id1: str, id2: str, playlist_df: pd.DataFrame) -> dict:
    """Returns a dictionary with the similarity of each audio feature for the 2 given tracks. Smaller numbers mean more similar."""
    similarities = {}
    
    # A list of features to compare:
    features = ['danceability', 'energy', 'instrumentalness', 'liveness', 'valence', 'loudness', 'tempo', 'key', 'mode']
    
    track1 = playlist_df.loc[id1] # variable containing all feature data from track 1
    track2 = playlist_df.loc[id2] # variable containing all feature data from track 2
    
    # Store all music feature data in dictionary form:
    # The important part of this code is converting everything to a number (not a string) that can be manipulated
    track1_features = {}
    track2_features = {}
    for feature in features:
        track1_features[feature] = track1.get(feature)
        track2_features[feature] = track2.get(feature)
    
    feature_similarities = {}
    for feature in features:
        if feature not in ['loudness', 'tempo', 'mode', 'key']:     # these features aren't stored as scalars, meaning their comparisons need to be handled differently
            # Compute the difference between the feature data for each track
            feature_similarities[feature] = abs(track1_features[feature] - track2_features[feature])
            
        if feature == 'loudness':
            feature_similarities[feature] = abs(track1_features[feature] - track2_features[feature]) * (1/60)  # loudness is stored -60 to 0, this converts the difference to a positive scalar
            # In rare cases, loudness falls outside the -60 to 0 range. If so, the code to scale down the difference between loudness to a scalar might result in a value over 1.
            # This code caps the maximum amount of difference at 1 - in other words, tracks that have over 60 dB of loudness difference are treated as having 60 dB of loudness difference.
            if feature_similarities[feature] > 1:
                feature_similarities[feature] = 1
                
        if feature == 'tempo':
            # Tempo is measured in BPM. This could be any positive integer, but most are within a range of 40-200 BPM.
            # For the purpose of comparison, tracks under 40 BPM are treated as having a BPM of 40. Tracks over 200 are treated as having a BPM of 200.
            # Thus, the maximum difference is 160, and the minimum is 0.
            # So to convert to a scalar, the difference between BPM is divided by 160.
            if track1_features[feature] < 40:
                track1_features[feature] = 40
            if track1_features[feature] > 200:
                track1_features[feature] = 200
            if track2_features[feature] < 40:
                track2_features[feature] = 40
            if track2_features[feature] > 200:
                track2_features[feature] = 200
            
            feature_similarities[feature] = abs(track1_features[feature] - track2_features[feature]) * (1/140)
            
        if feature == 'key':
            feature_similarities[feature] = compare_keys(track1_features['key'], track1_features['mode'], track2_features['key'], track2_features['mode'])
        
        if feature == 'mode':
            feature_similarities[feature] = float(abs(track1_features[feature] - track2_features[feature]))     # converted to float (from int) for uniformity, no practical effect
    
    return feature_similarities


def similarity_scalar_from_dict(feature_similarities: dict, **weights) -> float:
    """
    Returns a scalar from 0-1 that tells you how similar 2 songs are based on a mash-up of their audio features.
    By default, each audio feature is weighted equally in comparison (weight = 1). However, you can include a parameter for any audio feature you want.
    Setting this value to 0, for example, will give that feature no weight. Setting it to 4 will make that feature 4x as important as every other feature.
    Example usages:
        similarity_scalar(feature_similarities)
        similarity_scalar(feature_similarities, valence=4)
        similarity_scalar(feature_similarities, valence=4, mode=0, key=10)
    """
    
    # The scalar is computed as an average of all the similarities.
    total = 0                                                           # Stores the current sum of similarity differences
    total_weight = len(feature_similarities)                            # Stores the current sum of weights, by default = number of features * 1
    for similarity in feature_similarities.items():                     # Iterate through all audio features given (.items() iterates a tuple: (feature, similarity))
        try:                                                            # Act as if there's a given weight for this feature. Try to compute that.
            total += similarity[1] * weights[similarity[0]]             # Add this similarity times its special weight to the sum of similarities.
            total_weight += weights[similarity[0]]                      # Add the weight to the total weight. The sum of similarities is divided by this number.
        except KeyError:                                                # Base case: no weight was given for this feature
            total += similarity[1]                                      # Add the similarity for this feature to the total similarity variable
    
    # Divide the total similarity variable by the number of features
    # Changing weights of features essentially changes the number of features
    # In the data, 2 identical songs have a similarity average of 0.
    # However, it makes intuitive sense for similarity to be measured as 1 = identical, and 0 = completely different.
    # Hence, the value is inverted:
    return 1 - (total / total_weight)


def similarity_scalar(id1: str, id2: str, playlist_df: pd.DataFrame, **weights) -> float:
    """
    Overloads the similarity_scalar function to just take song ids and optional feature weights
    But since function overloading isn't a thing in Python (why???) they're actually different functions.
    """
    return similarity_scalar_from_dict(similarity_dict(id1, id2, playlist_df), **weights)

# Since we already wrote a function that returns a dictionary associating a song title with its ID, we'll use that
song_ids = get_song_ids(playlist_tracks)
# We need to analyze the playlist:
playlist_df = analyze_playlist(creator_id, playlist_id, sp)
# Compare 2 songs with matching keys based on circle of fifths:
comp1 = similarity_dict(song_ids['Fucker'], song_ids['Jerk'], playlist_df)
# Compare a song to itself:
comp2 = similarity_dict(song_ids['Dumb'], song_ids['Dumb'], playlist_df)

# See comparison breakdowns:
print(comp1)
print(comp2)

# See similarity scalars:
print(similarity_scalar(song_ids['Fucker'], song_ids['Jerk'], playlist_df))     # A scalar for comp1 from before
print(similarity_scalar(song_ids['Fucker'], song_ids['Jerk'], playlist_df, instrumentalness=2000))      # Weight instrumentalness 2000x more important than everything else.
print(similarity_scalar(song_ids['Dumb'], song_ids['Dumb'], playlist_df))       # A scalar for comp2 from before

/var/folders/24/qxr9c06j19gcvdhrn3t51m000000gn/T/ipykernel_88185/3211627563.py:21: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  playlist_df = pd.concat([playlist_df, track_df], ignore_index=False) # changed ignore_index from True to False


{'danceability': 0.279, 'energy': 0.05399999999999994, 'instrumentalness': 0.000164, 'liveness': 0.31200000000000006, 'valence': 0.07500000000000001, 'loudness': 0.03184999999999999, 'tempo': 0.5792214285714287, 'key': 0.0, 'mode': 1.0}
{'danceability': 0.0, 'energy': 0.0, 'instrumentalness': 0.0, 'liveness': 0.0, 'valence': 0.0, 'loudness': 0.0, 'tempo': 0.0, 'key': 0.0, 'mode': 0.0}
0.7409738412698412
0.9986764203939416
1.0


## What does any of this mean?

To get a sense of how this working so far, I want to compare songs that I think are similar. For example, Green Day's Jackass and Pat Benatar's Heartbreaker seem like they would be ~fairly~ similar based on cursory listens. What does the function say?

In [20]:
similarity_scalar(song_ids['Jackass'], song_ids['Heartbreaker'], playlist_df)

0.6309943982010582

0.63! That seems to makes sense - they might be correlated, but not too strongly. What if I de-emphasize some of the features I think might be causing them to be more different, and emphasize more similar ones?

In [21]:
similarity_scalar(song_ids['Jackass'], song_ids['Heartbreaker'], playlist_df, energy=2, tempo=0.25, key=0, mode=0, valence=3)

0.861699469540518

Again, that seems to make sense - the songs are more similar when we focus on their subjectively similar characteristics.

What about two songs that have very different styles, Lil Wayne's Snitch and The Bryan Jonestown Massacre's F**ker?

In [22]:
similarity_scalar(song_ids['Snitch'], song_ids['Fucker'], playlist_df)

0.6176328042328042

I wouldn't have considered these two songs as similar as Jackass and Heartbreaker, yet they return a similar value. So either the default weighting needs to be adjusted (because the weights should be the same for every comparison!), or this isn't a very effective way to compare songs...

More to do!

## Things I still want to explore:
* Visualizing data that is collected
* Using numpy to analyze data
* Merging dataframes with Pandas, and the other powers of Pandas
* What are Louvain communities, and how can they be used?
* Using the NetworkX library
* Become more comfortable using Pandas directly - I felt it was necessary to translate the data into a dictionary to work with it efficiently